In [ ]:
import numpy as np
from pathlib import Path
import re
import scipy.io as sio 

In [ ]:
sio.loadmat('USC-HAD/Subject10/a1t2.mat')['sensor_readings'].shape

In [ ]:
def extract_label_from_filename(filename):
    match = re.search(r'a(\d+)t', filename)
    if match:
        return int(match.group(1))
    return None

p = Path('USC-HAD/Subject10/a1t2.mat')
p.stem

extract_label_from_filename(p.stem)

In [ ]:


dir_uschad = Path('USC-HAD')
max_length = -1

def extract_label_from_filename(filename):
    match = re.search(r'a(\d+)t', filename)
    if match:
        return int(match.group(1))
    return None

X_irregular_length = []
Y = []
subject = []

for file_path in dir_uschad.rglob('*.mat'):
    data = sio.loadmat(file_path)['sensor_readings']
    max_length = data.shape[0] if data.shape[0] > max_length else max_length
    X_irregular_length.append(data)
    Y.append(extract_label_from_filename(file_path.stem))


In [ ]:
n_samples = len(X_irregular_length)
n_ticks, n_directions = X_irregular_length[0].shape
X = np.zeros([n_samples, max_length, n_directions])
for i, x in enumerate(X_irregular_length):
    sample_size = x.shape[0]
    X[i, :sample_size, ...] = x

In [ ]:
np.savez(
    'USC-HAD.npz',
    **{
        'label_movement': Y,
        'inertial_data': X,
    }
)

In [ ]:
data = np.load('USC-HAD.npz')
data = {key: data[key] for key in data}

In [ ]:
data_sum = np.sum(np.absolute(data['inertial_data']), axis=1)

import matplotlib.pyplot as plt

fig, axs = plt.subplots(6, 1, figsize=(12, 10))

for i in range(6):
    max_value = np.max(data_sum[:, i])
    axs[i].set_ylim(0, max_value)
    axs[i].plot(data_sum[:, i])

In [ ]:
data['inertial_data'].shape